In [111]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random, numpy as np
import pandas as pd

torch.manual_seed(1)

In [112]:
#loading and prepping data
#initially only one action
dframe = pd.read_csv('./csv_data/action_1.csv')

In [128]:
dframe.loc[dframe['video_id'].isin([0,1,2])]

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,label,id,video_id
0,-0.382997,-0.419442,3.449989,-0.366909,-0.092619,3.443680,-0.353380,0.229542,3.427116,-0.391862,...,3.636719,-0.435790,-0.536338,3.280097,-0.364369,-0.491436,3.269750,1,72057594037944340,0
1,-0.383146,-0.419292,3.450006,-0.367569,-0.092003,3.443895,-0.353885,0.230300,3.427162,-0.391820,...,3.633053,-0.436031,-0.536649,3.281972,-0.358806,-0.471054,3.269975,1,72057594037944340,0
2,-0.385776,-0.421191,3.449611,-0.369506,-0.092775,3.443796,-0.354571,0.230189,3.426965,-0.403822,...,3.632370,-0.436489,-0.536484,3.286322,-0.358079,-0.470344,3.270202,1,72057594037944340,0
3,-0.385807,-0.421205,3.449582,-0.369576,-0.092714,3.443878,-0.354524,0.230369,3.427140,-0.403580,...,3.499778,-0.441701,-0.533234,3.278971,-0.360298,-0.476572,3.268953,1,72057594037944340,0
4,-0.357840,-0.420304,3.438846,-0.364956,-0.092426,3.442334,-0.354907,0.230391,3.427352,-0.405945,...,3.400878,-0.430001,-0.536492,3.278641,-0.358697,-0.471415,3.270685,1,72057594037944340,0
5,-0.317779,-0.424233,3.417407,-0.347579,-0.091598,3.435570,-0.354990,0.231382,3.428351,-0.413833,...,3.376484,-0.440098,-0.535446,3.284364,-0.360256,-0.475610,3.269022,1,72057594037944340,0
6,-0.340148,-0.420218,3.429176,-0.349129,-0.091376,3.437132,-0.355946,0.231430,3.430860,-0.419386,...,3.402198,-0.431560,-0.535082,3.279142,-0.358869,-0.482122,3.268943,1,72057594037944340,0
7,-0.326968,-0.420078,3.419758,-0.347450,-0.091232,3.436175,-0.357365,0.231702,3.432635,-0.423889,...,3.375357,-0.435553,-0.536029,3.278222,-0.357651,-0.469825,3.270531,1,72057594037944340,0
8,-0.327881,-0.426224,3.441594,-0.346586,-0.092981,3.442161,-0.357856,0.231570,3.434157,-0.423823,...,3.362528,-0.438622,-0.534724,3.280972,-0.359408,-0.476015,3.269827,1,72057594037944340,0
9,-0.375689,-0.436647,3.439764,-0.356871,-0.095212,3.445247,-0.358203,0.232436,3.436680,-0.425678,...,3.341025,-0.432623,-0.537754,3.283831,-0.357795,-0.465352,3.270250,1,72057594037944340,0


In [143]:
#making test and train split
def train_test_split(dframe):
    label = dframe.iloc[0,75]
    num_samples = len(dframe.iloc[:,:])
    video_ids = np.unique(dframe.iloc[:,-1].values)
    train_video_ids = video_ids[:-15]
    test_video_ids = video_ids[-15:]
    train_split = np.empty(len(train_video_ids), dtype=object)
    test_split = np.empty(len(test_video_ids), dtype=object)
    
    for idx,i in enumerate(train_video_ids):
        train_split[idx] = dframe.loc[dframe['video_id'] == i].values[:,0:75]
        mean_vec = np.mean(train_split[idx], axis=0)
        std_vec = np.std(train_split[idx], axis=0)
        train_split[idx] = (train_split[idx] - mean_vec)/std_vec
    
    for idx,i in enumerate(test_video_ids):
        test_split[idx] = dframe.loc[dframe['video_id'] == i].values[:,0:75]
        mean_vec = np.mean(test_split[idx], axis=0)
        std_vec = np.std(test_split[idx], axis=0)
        test_split[idx] = (test_split[idx] - mean_vec)/std_vec
    
    return (train_split, label), (test_split, label)
    #print(train_split[0])
(trainX, trainL), (testX, testL) = train_test_split(dframe)

In [161]:
a = trainX[0]
print(a.shape)
len(np.concatenate((a,np.zeros((1,75))), axis=0))

(46, 75)


47

In [169]:
def Data_gen( train_split, SEQ_LEN):
    while(True):
        X = train_split[0]
        label = train_split[1]
        databatch = random.sample(list(X), 1)[0]
        #print(databatch.shape)
        if len(databatch) > SEQ_LEN:
            databatch = databatch[0:SEQ_LEN]
        elif len(databatch) < SEQ_LEN:
            databatch = np.concatenate((databatch, np.zeros((SEQ_LEN - len(databatch), 75))))
        else:
            pass
        yield databatch,label

ACTd = Data_gen((trainX, trainL), 120)

In [170]:
next(ACTd)

(90, 75)


(array([[-1.68956199, -1.26268849, -1.67073665, ..., -0.38852241,
         -1.19856897, -0.96502583],
        [-1.70663262, -1.26271552, -1.671211  , ..., -0.86085688,
         -0.74202888, -0.06545738],
        [-1.70722949, -1.26215215, -1.66516306, ..., -0.17952811,
         -0.82079673, -1.08859399],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]), 1)

In [62]:
#some random semantics test data
num_data = 100
dataX = [autograd.Variable(torch.randn((1, 75))) for _ in range(num_data)]
dataY = [autograd.Variable(torch.LongTensor([random.randint(0,9)])) for _ in range(num_data)]
# print(dataY)
# print(dataX)

In [9]:
inp_dim=75
hidden_dim=75
lstm = nn.LSTM(inp_dim, hidden_dim) # Input dim is 3, output dim is 3
inputs = [autograd.Variable(torch.randn((1, inp_dim)))
          for _ in range(10)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (autograd.Variable(torch.randn(1, 1, hidden_dim)),
          autograd.Variable(torch.randn((1, 1, hidden_dim))))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (autograd.Variable(torch.randn(1, 1, hidden_dim)),
          autograd.Variable(torch.randn((1, 1, hidden_dim))))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.4422 -0.0836 -0.3623  0.0170  0.0153  0.3019  0.9204  0.0295  0.2799

Columns 9 to 17 
   0.0171  0.3507 -0.2216 -0.4838 -0.0374  0.1170  0.3147  0.2965 -0.6561

Columns 18 to 26 
  -0.0643  0.7508 -0.4947 -0.1613 -0.0253  0.2302  0.1747  0.4461  0.1309

Columns 27 to 35 
   0.0815 -0.1382 -0.5025  0.0445 -0.0946 -0.1964  0.1771 -0.2011  0.1233

Columns 36 to 44 
   0.1657 -0.1282 -0.0748 -0.3011  0.1242 -0.0762  0.3110 -0.0740  0.3555

Columns 45 to 53 
  -0.4821 -0.4017  0.0443  0.3157  0.5549 -0.2614  0.0037  0.2796  0.0886

Columns 54 to 62 
  -0.3237 -0.0264 -0.0998 -0.1732 -0.2605 -0.0618  0.2819 -0.0445  0.1246

Columns 63 to 71 
   0.1765  0.1309  0.1503  0.1623 -0.2204 -0.1555 -0.4385  0.1237  0.0111

Columns 72 to 74 
  -0.1999  0.1897 -0.5549

(1 ,.,.) = 

Columns 0 to 8 
   0.0849  0.0138 -0.5418 -0.1290 -0.1049  0.4493  0.2657  0.0803  0.2891

Columns 9 to 17 
   0.1018  0.0534  0.0810 -0.4659 -0.0364 -0.1796  0.2171 -

In [105]:
#action LSTM
class LSTMClassifier(nn.Module):
    def __init__(self, joints_dim, hidden_dim, label_size, batch_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.lstm = nn.LSTM(joints_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # the first is the hidden h
        # the second is the cell  c
        return (autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

    def forward(self, joints3d_vec):
#         embeds = self.word_embeddings(sentence)
#         x = embeds.view(len(sentence), self.batch_size , -1)
        x = torch.cat(joints3d_vec).view(len(joints3d_vec), self.batch_size, -1)
        #print(x.shape)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        #print(lstm_out.shape)
        y  = self.hidden2label(lstm_out[-1])
        #print(y.shape)
        log_probs = F.log_softmax(y, dim=1)
        #print(log_probs.shape)
        return log_probs

In [99]:
l = torch.cat(dataY)
#print(l)
F.cross_entropy(autograd.Variable(torch.rand(len(l), 10)),l).shape

torch.Size([1])

In [106]:
model0 = LSTMClassifier(75, 75, 10, 1)

In [107]:
model0.forward(dataX)

Variable containing:
-2.2865 -2.4691 -2.4013 -2.2449 -2.2166 -2.4454 -2.1285 -2.3059 -2.2104 -2.3735
[torch.FloatTensor of size 1x10]

In [108]:
#training function
def train(dataX, dataY, model):
    num_iter = 10
    for i in range(num_iter):
        d_in = dataX #todo make an iterator
        l_in = dataY #again iterator
        y_pred = model(d_in)
        #print(y_pred)
        loss = F.cross_entropy(y_pred, l_in[0])
        #print(loss)
        loss.backward()
        

In [109]:
train(dataX, l, model0)

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.